In [241]:
import pandas as pd
import gurobipy as gp
import math
import pickle as pkl
from datetime import datetime, timedelta

In [242]:
product_type = "chilled"
num_time_periods = 7

In [243]:
article_data = pd.read_csv("./data/article.csv")
article_data = article_data[(article_data['TEMPERATURE_ZONE'] == product_type)]
# article_data = article_data.head(10)
test_article = '3ef73cbe'
article_data = article_data[(article_data['ARTICLE_ID'] == test_article)]

In [244]:

if(product_type=="frozen"):
    warehouse_volume = 30
if(product_type=="chilled"):
    warehouse_volume = 250
if(product_type=="ambient"):
    warehouse_volume = 700
buffer_cost = 25
max_order_exceed_multiplier = 1.5

In [245]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [246]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [247]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu, minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [248]:
forecast_data = pd.read_csv('./data/sales_'+str(num_time_periods)+'.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]

In [249]:
# Create a new dataframe with all dates
all_dates_df = pd.DataFrame({'DATE': pd.date_range(start=min(forecast_data['DATE']), end=max(forecast_data['DATE']), freq='D')}).astype(str)

# Group the original dataframe by item
grouped = forecast_data.groupby('ARTICLE_ID')

# Initialize an empty list to store the new dataframes
new_dfs = []

# Loop over each group
for item, group_df in grouped:
    
    group_df['DATE'] = pd.to_datetime(group_df['DATE']).astype(str)

    # Merge the group dataframe with the all_dates dataframe
    merged_df = pd.merge(all_dates_df, group_df, on='DATE', how='outer')
    merged_df['ARTICLE_ID'] = item
    
    # Fill in missing values
    merged_df['PICKING_QUANTITY_CU'] = merged_df['PICKING_QUANTITY_CU'].fillna(0)
    
    # Sort by date and append to the list
    new_dfs.append(merged_df.sort_values('DATE'))
    
# Concatenate all new dataframes into a single dataframe
forecast_data = pd.concat(new_dfs)
time_periods = forecast_data['DATE'].unique()

time_indexes = [*range(len(time_periods))]
date_to_index = {time_periods[i]:[*range(len(time_periods))][i] for i in time_indexes}
index_to_date = {[*range(len(time_periods))][i]:time_periods[i] for i in time_indexes}

# demand = forecast_data.groupby('DATE').apply(lambda x: dict(zip(x['ARTICLE_ID'], x['PICKING_QUANTITY_CU']))).to_dict()
# demand = dict((date_to_index[key],value) for (key,value) in demand.items())

demand = forecast_data.groupby('ARTICLE_ID').apply(lambda x: dict(zip(x['DATE'], x['PICKING_QUANTITY_CU']))).to_dict()
for item in demand.keys():
    demand[item] = dict((date_to_index[key], value) for (key, value) in demand[item].items())

In [250]:
cost_dict = {
    'fixed':0,
    'per_tu':0,
    'clearance':0,
    'buffer':0,
    'max_order_penalty':0,
    'total':0
}

In [251]:
def getCost(orders, order_date, cu_per_tu_item, order_q, disposal_cost):
        cost = 0
        if (orders[order_date]* cu_per_tu_item) - order_q > 0:
            cost = ((orders[order_date] * cu_per_tu_item) - order_q) * disposal_cost
        return cost   

def getMaxOrderPenalty(orders, order_date, variable_cost, maximum_order_quantity_item):
    max_order = 0
    if orders[order_date] > maximum_order_quantity_item: 
        #max_order = ((orders[order_date] * cu_per_tu_item) - maximum_order_quantity_item) * max_order_exceed_multiplier * variable_cost
        max_order = ((orders[order_date]) - maximum_order_quantity_item) * max_order_exceed_multiplier * variable_cost 
    return max_order 

def getTotalCosts(orders, order_date, cu_per_tu_item, disposal_cost, variable_cost, maximum_order_quantity_tu_item, fixed_cost, order_q, expired_flag):
    tot_disposal = 0
    if expired_flag == True:
        tot_disposal = getCost(orders, order_date, cu_per_tu_item, order_q, disposal_cost)      
    tot_ordering = orders[order_date] * variable_cost
    tot_fixed = fixed_cost
    tot_max_order =  getMaxOrderPenalty(orders, order_date, variable_cost, maximum_order_quantity_tu_item)
    #print(item, tot_ordering, tot_fixed)
    return tot_disposal, tot_ordering, tot_fixed, tot_max_order


In [252]:
def silverMeal(item):
    item_demand = list(demand[item].values())
    fixed_cost = ordering_cost_fixed[item]
    variable_cost = ordering_cost_per_tu[item]
    cu_per_tu_item = cu_per_tu[item]
    disposal_cost = clearing_cost_per_cu[item]
    maximum_order_quantity_tu_item = maximum_order_quantity_tu[item]
    minimum_order_quantity_tu_item = minimum_order_quantity_tu[item]
    life = shelf_life[item]
    cost = 0
    tot_disposal = 0
    tot_fixed = 0
    tot_ordering = 0
    tot_max_order = 0
    penalty = 0
    orders = {}
    order_date = 0 
    run_complete = False
    obj_val = 0
    v = 0
    while(not run_complete):
  
        if (order_date %7 == 6):
            order_date = order_date + 1 
            continue   
        
        min_cost = float('inf')
        cum_demand = 0
        
        expired_flag = False

        for t in range(order_date, len(item_demand)):
            if(t%7==6 ):
                continue

            cum_demand += item_demand[t]
            order_q = cum_demand

            #if shelf life of an item exceeds the period
            if(t ==order_date+life):
                
                expired_flag = True
                if (t == len(item_demand)-1):
                    expired_flag = False
                order_q = cum_demand

                orders[order_date] = math.ceil(order_q/cu_per_tu_item)
                if (math.ceil(order_q/cu_per_tu_item) < minimum_order_quantity_tu_item):
                    order_q = minimum_order_quantity_tu_item
                    orders[order_date] = order_q

                if v>0 and order_q -v > 0 :
                    order_q = order_q - v
                    orders[order_date] = math.ceil(order_q/cu_per_tu_item)
                print(v, "excess remained at loop 1", cum_demand, order_date, order_q, t)
                #v = (math.ceil(order_q / cu_per_tu_item) * cu_per_tu_item) - cum_demand
                v = 0
                d, o, f, m = getTotalCosts(orders, order_date, cu_per_tu_item, disposal_cost, variable_cost, maximum_order_quantity_tu_item, fixed_cost, order_q, expired_flag)
                tot_disposal += d
                tot_ordering += o
                tot_fixed += f
                tot_max_order += m
                order_date = t + 1
                break

            cost = fixed_cost + variable_cost * math.ceil(order_q/cu_per_tu_item) 
            if(math.ceil(order_q/cu_per_tu_item) > maximum_order_quantity_tu_item):
                penalty = (math.ceil(order_q/cu_per_tu_item) - maximum_order_quantity_tu_item) * max_order_exceed_multiplier * variable_cost
            avg_cost = (cost + penalty) / (t-order_date+1)
            
            #if the average cost is less now than later
            if(avg_cost <= min_cost):
                min_cost = avg_cost
                if(t==len(item_demand)-1):
                    order_q = cum_demand
                    
                    orders[order_date] = math.ceil(order_q/cu_per_tu_item)

                    if (math.ceil(order_q/cu_per_tu_item) < minimum_order_quantity_tu_item):
                        order_q = minimum_order_quantity_tu_item
                        orders[order_date] = order_q
                    
                    if v>0  and  order_q - v > 0:
                        order_q = order_q - v
                        print(order_q)
                        orders[order_date] = math.ceil(order_q/cu_per_tu_item)
                    #print("loop 2")
                    #v = (math.ceil(order_q / cu_per_tu_item) * cu_per_tu_item) - cum_demand
                    v = (orders[order_date]  * cu_per_tu_item) - order_q 
                    
                    print(v, "excess remained at loop 2", cum_demand, order_date, order_q, t)
                    d, o, f, m = getTotalCosts(orders, order_date, cu_per_tu_item, disposal_cost, variable_cost, maximum_order_quantity_tu_item, fixed_cost, order_q, expired_flag)
                    tot_disposal += d
                    tot_ordering += o
                    tot_fixed += f
                    tot_max_order += m
                    run_complete = True
                    break

            # if end of time period is reached then just order till previoud demand           
            else:
                if(t==len(item_demand)-1):
                    order_q = cum_demand 
                else:
                    order_q = cum_demand - item_demand[t]
                
                orders[order_date] = math.ceil(order_q/cu_per_tu_item)

                if (math.ceil(order_q/cu_per_tu_item) < minimum_order_quantity_tu_item):
                    order_q = minimum_order_quantity_tu_item
                    orders[order_date] = order_q

                if v>0  and order_q - v > 0:
                    order_q = order_q - v
                    orders[order_date] = math.ceil(order_q/cu_per_tu_item)

                v = (orders[order_date]  * cu_per_tu_item) - order_q
                #print(orders[order_date] * cu_per_tu_item)
                
                print(v, "excess remained at loop 3", "cum demand = ", cum_demand, "orders = ", orders[order_date])
                print("order q = ", order_q )

                d, o, f, m = getTotalCosts(orders, order_date, cu_per_tu_item, disposal_cost, variable_cost, maximum_order_quantity_tu_item, fixed_cost, order_q, expired_flag)
                tot_disposal += d
                tot_ordering += o
                tot_fixed += f
                tot_max_order += m
                if(t==len(item_demand)-1):
                    order_date = t + 1
                    run_complete = True
                else:
                    order_date = t
                break
       
        obj_val += cost
        if(order_date > len(item_demand)-1):
            run_complete = True
            break
    tot_obj_val = tot_disposal +  tot_ordering + tot_fixed + tot_max_order     
    return (orders, tot_obj_val, tot_disposal, tot_ordering, tot_fixed, tot_max_order)

obj_val = 0 
vol = 0
tot_disposal = 0
tot_fixed = 0
tot_ordering = 0
tot_max_order = 0

schedule = {}
for item in demand.keys():
    orders, val, disposal, ordering, fixed, max_order_p = silverMeal(item) 
    obj_val += val
    tot_disposal += disposal
    tot_fixed += fixed
    tot_ordering += ordering
    tot_max_order += max_order_p
    for t in orders.keys():
        if orders[t] == 0:
            continue
        if item not in schedule.keys():
            schedule[item] = {}
        if t not in schedule[item].keys():
            schedule[item][t] = 0
        schedule[item][t] += orders[t]
            
    
#print(silverMeal('00ee8964'))
print(obj_val)
print('Ordering',tot_ordering)
print('Fixing ', tot_fixed)
print('Clearance ', tot_disposal)
print('Max Order ', tot_max_order)



0 excess remained at loop 1 14.0 0 14.0 2
9 excess remained at loop 3 cum demand =  3.0 orders =  1
order q =  1
6.0 excess remained at loop 2 4.0 4 4.0 5
140.5
Ordering 88.0
Fixing  44.099999999999994
Clearance  8.399999999999999
Max Order  0


802540.2499999999
Ordering 672191.3999999996
Fixing  106309.69999999995
Clearance  15579.59999999999
Max Order  8459.55

In [118]:
buf_cost = 0
vol = 0
time_indexes
volumes = {}

for time in range(len(time_indexes)):
    for item in schedule:
        if time in schedule[item].keys():
            vol += schedule[item][time] * cu_per_tu[item] * volume_per_cu[item]

    if time > 0:
        for item in demand.keys():
            if time in demand[item].keys():
                vol -= demand[item][time] * volume_per_cu[item]

    volumes[time] = vol
    if vol > warehouse_volume:

        buf_cost += (vol - warehouse_volume) * buffer_cost

buf_cost
obj_val = obj_val + buf_cost

In [119]:
cost_dict['total'] = obj_val
cost_dict['buffer'] = buf_cost
cost_dict['clearance'] = tot_disposal
cost_dict['fixed'] = tot_fixed
cost_dict['per_tu'] = tot_ordering
cost_dict['max_order_penalty'] = tot_max_order
cost_dict

{'fixed': 187.79999999999998,
 'per_tu': 1125.3,
 'clearance': 14.099999999999998,
 'buffer': 0,
 'max_order_penalty': 0,
 'total': 1327.2}

In [120]:
obj_val

1327.2

In [122]:
def neighbourhoodOperator1(schedule, obj_val):
    max_improvement = math.inf
    
    modified_schedule = {}
    day_wise_orders = {}

    buffer_penalty_tot = 0
    max_order_penalty_tot = 0
    fixed_ordering_cost_tot = 0 
    schedule_imp = schedule 
    count = 0
    iterator = 0
    MAX_ITER = 100
    MAX_REPEAT = 1
    prev_max_improvement = 0
    flag = True

    while count < MAX_REPEAT and iterator < MAX_ITER:
        cost = obj_val
        for item in schedule_imp:
            buffer_penalty = 0
            max_order_penalty = 0 
            modified_orders = {}
            # Check if the item has a second order date
            if len(schedule_imp[item]) > 1:
                order_days = list(schedule_imp[item].keys())
                for i in range (0,len(order_days)-1, 2):
                    day1 = i
                    day2 = i + 1
                    if len(schedule_imp[item])%2 != 0:
                        day3 = i + 2
                        
                        if (order_days[day3] - order_days[day2] <= shelf_life[item]):
                            flag = True 
                        else: 
                            modified_orders[order_days[day1]] = schedule_imp[item][order_days[day1]]
                            modified_orders[order_days[day2]] = schedule_imp[item][order_days[day2]]
                            modified_orders[order_days[day3]] = schedule_imp[item][order_days[day3]]
                            continue

                    if order_days[day2] - order_days[day1] <= shelf_life[item] and (len(time_indexes)-1 - order_days[day2] <= shelf_life[item]) and flag:
                        tot_orders = schedule_imp[item][order_days[day1]]+ schedule_imp[item][order_days[day2]]
                        vol = tot_orders * cu_per_tu[item] * volume_per_cu[item]
                        if vol + volumes[day1] > warehouse_volume:
                            buffer_penalty += buffer_cost * (vol + volumes[day1] - warehouse_volume)
                        if tot_orders > maximum_order_quantity_tu[item]:
                            max_order_penalty += ( tot_orders - maximum_order_quantity_tu[item]) * max_order_exceed_multiplier * ordering_cost_per_tu[item]
                        delta = - ordering_cost_fixed[item] + buffer_penalty + max_order_penalty 
                        if delta < 0:
                            fixed_ordering_cost_tot += ordering_cost_fixed[item] 
                            buffer_penalty_tot += buffer_penalty
                            max_order_penalty_tot += max_order_penalty
                            cost += delta
                            modified_orders[order_days[day1]] = tot_orders
                            volumes[order_days[day1]] += vol
                        else:
                            modified_orders[order_days[day1]] = schedule_imp[item][order_days[day1]]
                            modified_orders[order_days[day2]] = schedule_imp[item][order_days[day2]]

                    else:
                        modified_orders[order_days[day1]] = schedule_imp[item][order_days[day1]]
                        modified_orders[order_days[day2]] = schedule_imp[item][order_days[day2]]
                    
                    if len(schedule_imp[item])%2 != 0:
                        modified_orders[order_days[-1]] = schedule_imp[item][order_days[-1]]
                
                
                modified_schedule[item] = modified_orders
            
            else:
                if len(schedule[item]) == 1 : 
                    modified_schedule[item] = schedule_imp[item]
                else:
                    continue


            if cost!=0 and cost < max_improvement:
                max_improvement = cost
        
        print("Buffer penalty paid", buffer_penalty_tot)
        print("max order penalty paid", max_order_penalty_tot)
        print("fixed cost reduced by", fixed_ordering_cost_tot)
        schedule_imp = modified_schedule
        obj_val = max_improvement
        iterator +=1

        print(max_improvement)

        if(max_improvement==prev_max_improvement):
            count+=1
        prev_max_improvement = max_improvement
    
    return schedule_imp, obj_val, buffer_penalty_tot, max_order_penalty_tot, fixed_ordering_cost_tot

In [123]:
schedule_improved, obj_val, buffer_cost_delta, max_order_penalty_delta, fixed_cost_delta = neighbourhoodOperator1(schedule, obj_val)
cost_dict['buffer'] += buffer_cost_delta
cost_dict['max_order_penalty'] += max_order_penalty_delta
cost_dict['fixed'] -= fixed_cost_delta
cost_dict['total'] = obj_val
cost_dict

in here
in here
3ef73cbe
in here
in here
in here
e503d66c
in here
in here
Buffer penalty paid 0
max order penalty paid 0
fixed cost reduced by 44.3
1282.9
3ef73cbe
in here
in here
Buffer penalty paid 0
max order penalty paid 0
fixed cost reduced by 60.8
1266.4
3ef73cbe
in here
Buffer penalty paid 0
max order penalty paid 0
fixed cost reduced by 60.8
1266.4


{'fixed': 126.99999999999999,
 'per_tu': 1125.3,
 'clearance': 14.099999999999998,
 'buffer': 0,
 'max_order_penalty': 0,
 'total': 1266.4}

In [124]:
schedule_improved

{'060983da': {0: 32},
 '07e343d0': {0: 14},
 '3ef73cbe': {0: 2, 3: 1, 4: 1},
 'afa44c07': {0: 20},
 'd5b74c6e': {0: 35},
 'e503d66c': {0: 21},
 'f04b3d50': {0: 2, 3: 2}}

In [125]:
schedule

{'060983da': {0: 6, 2: 26},
 '07e343d0': {0: 5, 3: 9},
 '3ef73cbe': {0: 2, 3: 1, 4: 1},
 'afa44c07': {0: 10, 4: 10},
 'd5b74c6e': {0: 8, 2: 27},
 'e503d66c': {0: 7, 2: 4, 3: 10},
 'f04b3d50': {0: 2, 3: 2}}